In [1]:
import os, numpy as np, pandas, sklearn
import mido, rtmidi, rtmidi_
np.random.seed(333)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## NN libs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
import keras
from keras import regularizers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Activation, Conv1D, Conv2D, Dropout, Flatten
from keras.layers import Conv2DTranspose, Reshape, MaxPooling2D, UpSampling2D, UpSampling1D, MaxPooling1D
from keras.layers import LocallyConnected1D, LocallyConnected2D
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.callbacks import TensorBoard

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# local libs
import config
from data import data, midi
from utils import io

 Context :: namedtuple(
[ max_t = float
, dt = float
, n_instances = int
, note_length = int
, bpm = float
, tempo = float
, ticks_per_beat = int
]



In [4]:
n: int = 8
context, x_train, labels = data.import_data(data.init(),n)
print(x_train.shape)

Setting up params

 >> Context(max_t=3.0, dt=0.1, n_instances=30, note_length=0.03, bpm=120.0, tempo=500000, ticks_per_beat=480)
Importing midi-data

[INFO] :
 |  reading file: ../datasets/examples/01 16th Snare.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Cym.mid
[INFO] :
 |  reading file: ../datasets/examples/01 8th Hat.mid
[INFO] :
 |  reading file: ../datasets/examples/02 8th Cym.mid
[INFO] :
 |  reading file: ../datasets/examples/114_LetsDance_DavidBowie1.mid
[INFO] :
 |  reading file: ../datasets/examples/127_SweetChildOMine_GunsAndRoses1.mid
[INFO] :
 |  reading file: ../datasets/examples/92_GiveItAway_TheRedHotChiliPeppers.mid
[INFO] :
 |  reading file: ../datasets/examples/94_Creep_Radiohead2.mid

Encoding midi-data
 [<midi file '../datasets/examples/01 16th Snare.mid' type 0, 1 tracks, 182 messages>, <midi file '../datasets/examples/01 8th Cym.mid' type 0, 1 tracks, 68 messages>, <midi file '../datasets/examples/01 8th Hat.mid' type 0, 1 tracks, 60 messages>, <m

# Autoencoder
Input -> hidden layer -> output (=input)
Hidden layer has few dimension. Latent space of this layer should produce automatic categorization.

In [5]:
y_train = x_train
n_samples = x_train[0]
input_shape = x_train.shape[1:] # shape of a single sample
output_shape = y_train.shape[1:]
# output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
hidden_layer_length = 100
x_train.shape

(8, 30, 4)

In [6]:
y_train.shape

(8, 30, 4)

In [7]:
output_shape

(30, 4)

In [8]:
y_train = x_train
n_samples = x_train[0]
input_shape = x_train.shape[1:] # shape of a single sample
output_shape = y_train.shape[1:]
# output_length = y_train.shape[1]
# output_length = (y_train[0]).shape[0] # = length of an individual label
hidden_layer_length = 100
timesteps = input_shape[0]
input_length = input_shape[1]
x_train.shape

(8, 30, 4)

In [17]:
inputs = Input(shape=(timesteps, input_length))
x = inputs
x = Dense(127, activation='relu')(x)
encoded = LSTM(hidden_layer_length)(x)
dims = keras.backend.int_shape(encoded)
print('dims =', dims, hidden_layer_length)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_length, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
encoder.summary()
sequence_autoencoder.summary()

dims = (None, 100) 100
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 30, 4)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 30, 127)           635       
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               91200     
Total params: 91,835
Trainable params: 91,835
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 30, 4)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 30, 127)           635       
_________________________________________________

In [10]:
metrics = ['accuracy','mse','categorical_crossentropy']
loss = 'binary_crossentropy'
optimizer = 'adadelta'
# learning_rate = 0.001
# optimizer = optimizer = Adam(lr=learning_rate)
sequence_autoencoder.compile(optimizer=optimizer, loss=loss, metrics=metrics)
encoder.compile(optimizer='adadelta', loss=loss, metrics=metrics)

In [11]:
batch_size = 3
# n epochs = n iterations over all the training data
epochs = 15

In [12]:
sequence_autoencoder.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_log_dir)])
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
#           validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

KeyboardInterrupt: 

In [ ]:
encode = encoder

In [ ]:
result = encode.predict(x_train)
n = result.shape[0]
result.shape

In [ ]:
# scale/normalize data, based on features
# axis determines whether the data is scaled based on the 'global' features
# axis = 0: max per feature (between instances)
# axis = 1: max per instance (ignore features)
# e.g. np.arange(9).reshape(3,3).max(axis=0)

a = preprocessing.minmax_scale(np.arange(9).reshape(3,3), axis=0)
a

In [ ]:
a[:,1].mean()

In [ ]:
plt.plot(result[:,0])

In [ ]:
normalized = preprocessing.normalize(result, axis=0)
# normalized = preprocessing.minmax_scale(result, axis=0)
# normalized = preprocessing.robust_scale(result, axis=0, quantile_range=(25, 75))
normalized[:,2]

In [ ]:
result[:,2].mean()

In [ ]:
normalized[0,:].mean()

In [ ]:
data = normalized
data.shape

In [ ]:
data[:,2]

In [ ]:
data[:2, 0]

In [ ]:
# plt.scatter(latent[:,0], latent[:,1], c=y_test[:num], alpha=0.1)
# plt.scatter(result[:,0],result[:,1])
plt.scatter(data[:3, 0], data[:3, 1], data='o', c=[0.4]*3)
plt.scatter(data[4:, 0], data[4:, 1], data='^', c=[0.5]*3)

In [ ]:
x_dim = 0
y_dim = 1
c_dim = 2
plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    data[:, x_dim], data[:, y_dim], marker='o', c=data[:, c_dim], alpha=0.9, s=[50 for _ in range(n)],
    cmap=plt.get_cmap('Spectral'))

for label, x, y in zip(labels, data[:, x_dim], data[:, y_dim]):
    plt.annotate(
        label,
        xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='right', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

plt.show()

In [ ]:
originals = result[0:2,]
new = originals.mean(axis=0)
arr = decode.predict(new)

In [ ]:
midis = [midi.decode_track(context, track) for track in arr]